### doc_entity
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E ENTIDADES (PESSOAS). TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

In [171]:
import subprocess
import sys
import os
import pickle
import re
import numpy as np
import pandas as pd
from IPython.display import clear_output
# import getpass
# import pymysql
import sqlite3
import collections

In [38]:
if os.name == 'nt':
    encoding_type = 'utf-8'
else: 
    encoding_type = 'ISO-8859-1'

In [62]:
path = "../../"
path_inputs = os.path.join('..','..','textfiles-corrected-ignorecase')
outputs = os.path.join("..","outputs")
inputs = os.path.join("..", "inputs")

In [5]:
WORD_CLASSES = {'N':'Nouns', 
                'PROP':'Proper nouns', 
                'SPEC': 'Specifiers', 
                'DET': 'Determiners',
                'PERS':'Personal pronouns', 
                'ADJ':'Adjectives',
                'ADV':'Adverbs', 
                'V':'Verbs', 
                'NUM':'Numerals', 
                'PRP':'Preposition',
                'KS':'Subordinating conjunctions',
                'KC':'Coordinationg conjunctions',
                'IN':'Interjections',
                'EC':'Hyphen-separated prefix',
                'BL': 'Blank Line',
                'ES': 'End of Sentence',
                'NW': 'Non Word'}

INF_TAGS = {'M':'Male gender', 
            'F':'Female gender', 
            'M/F':'Neutral gender',
            'S':'Singular number', 
            'P':'Plural number', 
            'S/P':'Neutral number',
            'NOM':'Nominative case', 
            'ACC':'Accusative case', 
            'DAT':'Dative case',
            'PIV':'Prepositive case', 
            'ACC/DAT':'Accusative-Dative case', 
            'DAT':'Nominative-Prepositive case', 
            '1':'First person', 
            '2':'Second person', 
            '3':'Third person',
            '1S':'First person singular', 
            '2S':'Second person singular',
            '3S':'Third person singular',
            '1P':'First person plural', 
            '2P':'Second person plural',
            '3P':'Third person plural',
            '1/3S':'First or Third person singular', 
            '0/1/3S':'Impersonal or First or Third person singular',
            'PR':'presente simples',
            'IMPF':'preterito imperfeito', 
            'PS':'preterito perfeito',
            'MQP':'preterito mais-que-perfeito', 
            'FUT':'futuro do presente', 
            'COND':'futuro do preterito',
            'IND':'indicativo', 
            'SUBJ':'subjuntivo', 
            'IMP':'imperativo', 
            'VFIN':'Verbo Finito',
            'INF':'infinitivo', 
            'PCP':'participio', 
            'GER':'gerundio'}

SYN_TAGS = {'@SUBJ>':'subject', 
            '@<SUBJ':'subject',
            '@ACC>':'accusative direct object',
            '@<ACC':'accusative direct object', 
            '@DAT>':'dative object only pronominal', 
            '@<DAT':'dative object only pronominal',
            '@PIV>':'prepositional indirect object', 
            '@<PIV':'prepositional indirect object',
            '@ADVS> / @SA>':'adverbial object (place, time, duration, quantity), subject-related',
            '@<ADVS / @<SA':'adverbial object (place, time, duration, quantity), subject-related',
            '@ADVO> / @OA>':'adverbial object object-related',
            '@<ADVO / @<OA':'adverbial object object-related',
            '@SC>':'subject predicative',
            '@<SC':'subject predicative',
            '@OC>':'object predicative',
            '@<OC':'object predicative',
            '@ADVL>':'adverbial',
            '@<ADVL':'adverbial',
            '@PASS>':'agent of passive',
            '@<PASS':'agent of passive', #All above clause arguments attach to the nearest main verb to the left [<] or right [>]
            '@ADVL':'free adverbial phrase in non-sentence expression',
            '@NPHR':'free noun phrase in non-sentence expression without verbs',
            '@VOK':'vocative',
            '@>N':'prenominal adject', 
            '@N<':'postnominal adject', #both last attaches to the nearest NP-head that is not an adnominal itself
            '@N<PRED':'postnominal in-group predicative', #or predicate in small clause introduced by com/sem',
            '@APP':'identifying apposition',
            '@>A':'prepositioned adverbial adject', #attaches to the nearest ADJ/PCP/ADV or attributive used N to the right 
            '@A<': 'postpositioned adverbial adject', #or dependent/argument of attributive participle (with function tag attached
            '@PRED>': 'forward free predicative', #refers to the following @SUBJ, even when this is incorporated in the VP 
            '@<PRED': 'backward free predicative', #refers to the nearest NP-head to the left, or to the nearest @SUBJ to the left
            '@P<': 'argument of preposition', 
            '@S<': 'sentence anaphor', 
            '@FAUX': 'finite auxiliary',
            '@FMV': 'finite main verb', 
            '@IAUX': 'infinite auxiliary', 
            '@IMV': 'infinite main verb', 
            '@PRT-AUX<': 'verb chain particle',
            '@CO': 'coordinating conjunction', 
            '@SUB': 'subordinating conjunction', 
            '@KOMP<': 'argument of comparative',
            '@COM': 'direct comparator without preceding comparative',
            '@PRD': 'role predicator',
            '@FOC>': 'focus marker',
            '@<FOC': 'focus marker',
            '@TOP': 'topic constituent',
            '@#FS-': 'finite subclause', #combines with clausal role and intraclausal word tag, e.g.@#FS-<ACC @SUB for "não acredito que seja verdade") 
            '@#ICL-': 'infinite subclause', #combines with clausal role and intraclausal word tag, e.g. @#ICL-SUBJ> @IMV in "consertar um relógio não é fácil") 
            '@#ICL-AUX<': 'argument verb in verb chain', #refers to preceding auxiliary (the verb chain sequence @FAUX - @#ICL-AUX< is used, where both verbs have the same subject, @FMV - @#ICL-<ACC is used where the subjects are different) 
            '@#AS-': 'averbal subclause', #combines with clausal role and intraclausal word tag, e.g. @#AS-<ADVL @ADVL> in "ajudou onde possível") 
            '@AS<': 'argument of complementiser in averbal subclause'}

DEP_MOD_BUG = {'ADJ0/1/3S': 'ADJ 0/1/3S',
               'ADJ1': 'ADJ 1',
               'ADJ1/3S': 'ADJ 1/3S',
               'ADJ1P': 'ADJ 1P',
               'ADJ1S': 'ADJ 1S',
               'ADJ2': 'ADJ 2',
               'ADJ2P': 'ADJ 2P',
               'ADJ2S': 'ADJ 2S',
               'ADJ3': 'ADJ 3',
               'ADJ3P': 'ADJ 3P',
               'ADJ3S': 'ADJ 3S',
               'ADJACC': 'ADJ ACC',
               'ADJACC/DAT': 'ADJ ACC/DAT',
               'ADJCOND': 'ADJ COND',
               'ADJDAT': 'ADJ DAT',
               'ADJF': 'ADJ F',
               'ADJFUT': 'ADJ FUT',
               'ADJGER': 'ADJ GER',
               'ADJIMP': 'ADJ IMP',
               'ADJIMPF': 'ADJ IMPF',
               'ADJIND': 'ADJ IND',
               'ADJINF': 'ADJ INF',
               'ADJM': 'ADJ M',
               'ADJM/F': 'ADJ M/F',
               'ADJMQP': 'ADJ MQP',
               'ADJNOM': 'ADJ NOM',
               'ADJP': 'ADJ P',
               'ADJPCP': 'ADJ PCP',
               'ADJPIV': 'ADJ PIV',
               'ADJPR': 'ADJ PR',
               'ADJPS': 'ADJ PS',
               'ADJS': 'ADJ S',
               'ADJS/P': 'ADJ S/P',
               'ADJSUBJ': 'ADJ SUBJ',
               'ADJVFIN': 'ADJ VFIN',
               'ADV0/1/3S': 'ADV 0/1/3S',
               'ADV1': 'ADV 1',
               'ADV1/3S': 'ADV 1/3S',
               'ADV1P': 'ADV 1P',
               'ADV1S': 'ADV 1S',
               'ADV2': 'ADV 2',
               'ADV2P': 'ADV 2P',
               'ADV2S': 'ADV 2S',
               'ADV3': 'ADV 3',
               'ADV3P': 'ADV 3P',
               'ADV3S': 'ADV 3S',
               'ADVACC': 'ADV ACC',
               'ADVACC/DAT': 'ADV ACC/DAT',
               'ADVCOND': 'ADV COND',
               'ADVDAT': 'ADV DAT',
               'ADVF': 'ADV F',
               'ADVFUT': 'ADV FUT',
               'ADVGER': 'ADV GER',
               'ADVIMP': 'ADV IMP',
               'ADVIMPF': 'ADV IMPF',
               'ADVIND': 'ADV IND',
               'ADVINF': 'ADV INF',
               'ADVM': 'ADV M',
               'ADVM/F': 'ADV M/F',
               'ADVMQP': 'ADV MQP',
               'ADVNOM': 'ADV NOM',
               'ADVP': 'ADV P',
               'ADVPCP': 'ADV PCP',
               'ADVPIV': 'ADV PIV',
               'ADVPR': 'ADV PR',
               'ADVPS': 'ADV PS',
               'ADVS': 'ADV S',
               'ADVS/P': 'ADV S/P',
               'ADVSUBJ': 'ADV SUBJ',
               'ADVVFIN': 'ADV VFIN',
               'DET0/1/3S': 'DET 0/1/3S',
               'DET1': 'DET 1',
               'DET1/3S': 'DET 1/3S',
               'DET1P': 'DET 1P',
               'DET1S': 'DET 1S',
               'DET2': 'DET 2',
               'DET2P': 'DET 2P',
               'DET2S': 'DET 2S',
               'DET3': 'DET 3',
               'DET3P': 'DET 3P',
               'DET3S': 'DET 3S',
               'DETACC': 'DET ACC',
               'DETACC/DAT': 'DET ACC/DAT',
               'DETCOND': 'DET COND',
               'DETDAT': 'DET DAT',
               'DETF': 'DET F',
               'DETFUT': 'DET FUT',
               'DETGER': 'DET GER',
               'DETIMP': 'DET IMP',
               'DETIMPF': 'DET IMPF',
               'DETIND': 'DET IND',
               'DETINF': 'DET INF',
               'DETM': 'DET M',
               'DETM/F': 'DET M/F',
               'DETMQP': 'DET MQP',
               'DETNOM': 'DET NOM',
               'DETP': 'DET P',
               'DETPCP': 'DET PCP',
               'DETPIV': 'DET PIV',
               'DETPR': 'DET PR',
               'DETPS': 'DET PS',
               'DETS': 'DET S',
               'DETS/P': 'DET S/P',
               'DETSUBJ': 'DET SUBJ',
               'DETVFIN': 'DET VFIN',
               'EC0/1/3S': 'EC 0/1/3S',
               'EC1': 'EC 1',
               'EC1/3S': 'EC 1/3S',
               'EC1P': 'EC 1P',
               'EC1S': 'EC 1S',
               'EC2': 'EC 2',
               'EC2P': 'EC 2P',
               'EC2S': 'EC 2S',
               'EC3': 'EC 3',
               'EC3P': 'EC 3P',
               'EC3S': 'EC 3S',
               'ECACC': 'EC ACC',
               'ECACC/DAT': 'EC ACC/DAT',
               'ECCOND': 'EC COND',
               'ECDAT': 'EC DAT',
               'ECF': 'EC F',
               'ECFUT': 'EC FUT',
               'ECGER': 'EC GER',
               'ECIMP': 'EC IMP',
               'ECIMPF': 'EC IMPF',
               'ECIND': 'EC IND',
               'ECINF': 'EC INF',
               'ECM': 'EC M',
               'ECM/F': 'EC M/F',
               'ECMQP': 'EC MQP',
               'ECNOM': 'EC NOM',
               'ECP': 'EC P',
               'ECPCP': 'EC PCP',
               'ECPIV': 'EC PIV',
               'ECPR': 'EC PR',
               'ECPS': 'EC PS',
               'ECS': 'EC S',
               'ECS/P': 'EC S/P',
               'ECSUBJ': 'EC SUBJ',
               'ECVFIN': 'EC VFIN',
               'IN0/1/3S': 'IN 0/1/3S',
               'IN1': 'IN 1',
               'IN1/3S': 'IN 1/3S',
               'IN1P': 'IN 1P',
               'IN1S': 'IN 1S',
               'IN2': 'IN 2',
               'IN2P': 'IN 2P',
               'IN2S': 'IN 2S',
               'IN3': 'IN 3',
               'IN3P': 'IN 3P',
               'IN3S': 'IN 3S',
               'INACC': 'IN ACC',
               'INACC/DAT': 'IN ACC/DAT',
               'INCOND': 'IN COND',
               'INDAT': 'IN DAT',
               'INF': 'IN F',
               'INFUT': 'IN FUT',
               'INGER': 'IN GER',
               'INIMP': 'IN IMP',
               'INIMPF': 'IN IMPF',
               'ININD': 'IN IND',
               'ININF': 'IN INF',
               'INM': 'IN M',
               'INM/F': 'IN M/F',
               'INMQP': 'IN MQP',
               'INNOM': 'IN NOM',
               'INP': 'IN P',
               'INPCP': 'IN PCP',
               'INPIV': 'IN PIV',
               'INPR': 'IN PR',
               'INPS': 'IN PS',
               'INS': 'IN S',
               'INS/P': 'IN S/P',
               'INSUBJ': 'IN SUBJ',
               'INVFIN': 'IN VFIN',
               'KC0/1/3S': 'KC 0/1/3S',
               'KC1': 'KC 1',
               'KC1/3S': 'KC 1/3S',
               'KC1P': 'KC 1P',
               'KC1S': 'KC 1S',
               'KC2': 'KC 2',
               'KC2P': 'KC 2P',
               'KC2S': 'KC 2S',
               'KC3': 'KC 3',
               'KC3P': 'KC 3P',
               'KC3S': 'KC 3S',
               'KCACC': 'KC ACC',
               'KCACC/DAT': 'KC ACC/DAT',
               'KCCOND': 'KC COND',
               'KCDAT': 'KC DAT',
               'KCF': 'KC F',
               'KCFUT': 'KC FUT',
               'KCGER': 'KC GER',
               'KCIMP': 'KC IMP',
               'KCIMPF': 'KC IMPF',
               'KCIND': 'KC IND',
               'KCINF': 'KC INF',
               'KCM': 'KC M',
               'KCM/F': 'KC M/F',
               'KCMQP': 'KC MQP',
               'KCNOM': 'KC NOM',
               'KCP': 'KC P',
               'KCPCP': 'KC PCP',
               'KCPIV': 'KC PIV',
               'KCPR': 'KC PR',
               'KCPS': 'KC PS',
               'KCS': 'KC S',
               'KCS/P': 'KC S/P',
               'KCSUBJ': 'KC SUBJ',
               'KCVFIN': 'KC VFIN',
               'KS0/1/3S': 'KS 0/1/3S',
               'KS1': 'KS 1',
               'KS1/3S': 'KS 1/3S',
               'KS1P': 'KS 1P',
               'KS1S': 'KS 1S',
               'KS2': 'KS 2',
               'KS2P': 'KS 2P',
               'KS2S': 'KS 2S',
               'KS3': 'KS 3',
               'KS3P': 'KS 3P',
               'KS3S': 'KS 3S',
               'KSACC': 'KS ACC',
               'KSACC/DAT': 'KS ACC/DAT',
               'KSCOND': 'KS COND',
               'KSDAT': 'KS DAT',
               'KSF': 'KS F',
               'KSFUT': 'KS FUT',
               'KSGER': 'KS GER',
               'KSIMP': 'KS IMP',
               'KSIMPF': 'KS IMPF',
               'KSIND': 'KS IND',
               'KSINF': 'KS INF',
               'KSM': 'KS M',
               'KSM/F': 'KS M/F',
               'KSMQP': 'KS MQP',
               'KSNOM': 'KS NOM',
               'KSP': 'KS P',
               'KSPCP': 'KS PCP',
               'KSPIV': 'KS PIV',
               'KSPR': 'KS PR',
               'KSPS': 'KS PS',
               'KSS': 'KS S',
               'KSS/P': 'KS S/P',
               'KSSUBJ': 'KS SUBJ',
               'KSVFIN': 'KS VFIN',
               'N0/1/3S': 'N 0/1/3S',
               'N1': 'N 1',
               'N1/3S': 'N 1/3S',
               'N1P': 'N 1P',
               'N1S': 'N 1S',
               'N2': 'N 2',
               'N2P': 'N 2P',
               'N2S': 'N 2S',
               'N3': 'N 3',
               'N3P': 'N 3P',
               'N3S': 'N 3S',
               'NACC': 'N ACC',
               'NACC/DAT': 'N ACC/DAT',
               'NCOND': 'N COND',
               'NDAT': 'N DAT',
               'NF': 'N F',
               'NFUT': 'N FUT',
               'NGER': 'N GER',
               'NIMP': 'N IMP',
               'NIMPF': 'N IMPF',
               'NIND': 'N IND',
               'NINF': 'N INF',
               'NM': 'N M',
               'NM/F': 'N M/F',
               'NMQP': 'N MQP',
               'NNOM': 'N NOM',
               'NP': 'N P',
               'NPCP': 'N PCP',
               'NPIV': 'N PIV',
               'NPR': 'N PR',
               'NPS': 'N PS',
               'NS': 'N S',
               'NS/P': 'N S/P',
               'NSUBJ': 'N SUBJ',
               'NUM0/1/3S': 'NUM 0/1/3S',
               'NUM1': 'NUM 1',
               'NUM1/3S': 'NUM 1/3S',
               'NUM1P': 'NUM 1P',
               'NUM1S': 'NUM 1S',
               'NUM2': 'NUM 2',
               'NUM2P': 'NUM 2P',
               'NUM2S': 'NUM 2S',
               'NUM3': 'NUM 3',
               'NUM3P': 'NUM 3P',
               'NUM3S': 'NUM 3S',
               'NUMACC': 'NUM ACC',
               'NUMACC/DAT': 'NUM ACC/DAT',
               'NUMCOND': 'NUM COND',
               'NUMDAT': 'NUM DAT',
               'NUMF': 'NUM F',
               'NUMFUT': 'NUM FUT',
               'NUMGER': 'NUM GER',
               'NUMIMP': 'NUM IMP',
               'NUMIMPF': 'NUM IMPF',
               'NUMIND': 'NUM IND',
               'NUMINF': 'NUM INF',
               'NUMM': 'NUM M',
               'NUMM/F': 'NUM M/F',
               'NUMMQP': 'NUM MQP',
               'NUMNOM': 'NUM NOM',
               'NUMP': 'NUM P',
               'NUMPCP': 'NUM PCP',
               'NUMPIV': 'NUM PIV',
               'NUMPR': 'NUM PR',
               'NUMPS': 'NUM PS',
               'NUMS': 'NUM S',
               'NUMS/P': 'NUM S/P',
               'NUMSUBJ': 'NUM SUBJ',
               'NUMVFIN': 'NUM VFIN',
               'NVFIN': 'N VFIN',
               'PERS0/1/3S': 'PERS 0/1/3S',
               'PERS1': 'PERS 1',
               'PERS1/3S': 'PERS 1/3S',
               'PERS1P': 'PERS 1P',
               'PERS1S': 'PERS 1S',
               'PERS2': 'PERS 2',
               'PERS2P': 'PERS 2P',
               'PERS2S': 'PERS 2S',
               'PERS3': 'PERS 3',
               'PERS3P': 'PERS 3P',
               'PERS3S': 'PERS 3S',
               'PERSACC': 'PERS ACC',
               'PERSACC/DAT': 'PERS ACC/DAT',
               'PERSCOND': 'PERS COND',
               'PERSDAT': 'PERS DAT',
               'PERSF': 'PERS F',
               'PERSFUT': 'PERS FUT',
               'PERSGER': 'PERS GER',
               'PERSIMP': 'PERS IMP',
               'PERSIMPF': 'PERS IMPF',
               'PERSIND': 'PERS IND',
               'PERSINF': 'PERS INF',
               'PERSM': 'PERS M',
               'PERSM/F': 'PERS M/F',
               'PERSMQP': 'PERS MQP',
               'PERSNOM': 'PERS NOM',
               'PERSP': 'PERS P',
               'PERSPCP': 'PERS PCP',
               'PERSPIV': 'PERS PIV',
               'PERSPR': 'PERS PR',
               'PERSPS': 'PERS PS',
               'PERSS': 'PERS S',
               'PERSS/P': 'PERS S/P',
               'PERSSUBJ': 'PERS SUBJ',
               'PERSVFIN': 'PERS VFIN',
               'PROP0/1/3S': 'PROP 0/1/3S',
               'PROP1': 'PROP 1',
               'PROP1/3S': 'PROP 1/3S',
               'PROP1P': 'PROP 1P',
               'PROP1S': 'PROP 1S',
               'PROP2': 'PROP 2',
               'PROP2P': 'PROP 2P',
               'PROP2S': 'PROP 2S',
               'PROP3': 'PROP 3',
               'PROP3P': 'PROP 3P',
               'PROP3S': 'PROP 3S',
               'PROPACC': 'PROP ACC',
               'PROPACC/DAT': 'PROP ACC/DAT',
               'PROPCOND': 'PROP COND',
               'PROPDAT': 'PROP DAT',
               'PROPF': 'PROP F',
               'PROPFUT': 'PROP FUT',
               'PROPGER': 'PROP GER',
               'PROPIMP': 'PROP IMP',
               'PROPIMPF': 'PROP IMPF',
               'PROPIND': 'PROP IND',
               'PROPINF': 'PROP INF',
               'PROPM': 'PROP M',
               'PROPM/F': 'PROP M/F',
               'PROPMQP': 'PROP MQP',
               'PROPNOM': 'PROP NOM',
               'PROPP': 'PROP P',
               'PROPPCP': 'PROP PCP',
               'PROPPIV': 'PROP PIV',
               'PROPPR': 'PROP PR',
               'PROPPS': 'PROP PS',
               'PROPS': 'PROP S',
               'PROPS/P': 'PROP S/P',
               'PROPSUBJ': 'PROP SUBJ',
               'PROPVFIN': 'PROP VFIN',
               'PRP0/1/3S': 'PRP 0/1/3S',
               'PRP1': 'PRP 1',
               'PRP1/3S': 'PRP 1/3S',
               'PRP1P': 'PRP 1P',
               'PRP1S': 'PRP 1S',
               'PRP2': 'PRP 2',
               'PRP2P': 'PRP 2P',
               'PRP2S': 'PRP 2S',
               'PRP3': 'PRP 3',
               'PRP3P': 'PRP 3P',
               'PRP3S': 'PRP 3S',
               'PRPACC': 'PRP ACC',
               'PRPACC/DAT': 'PRP ACC/DAT',
               'PRPCOND': 'PRP COND',
               'PRPDAT': 'PRP DAT',
               'PRPF': 'PRP F',
               'PRPFUT': 'PRP FUT',
               'PRPGER': 'PRP GER',
               'PRPIMP': 'PRP IMP',
               'PRPIMPF': 'PRP IMPF',
               'PRPIND': 'PRP IND',
               'PRPINF': 'PRP INF',
               'PRPM': 'PRP M',
               'PRPM/F': 'PRP M/F',
               'PRPMQP': 'PRP MQP',
               'PRPNOM': 'PRP NOM',
               'PRPP': 'PRP P',
               'PRPPCP': 'PRP PCP',
               'PRPPIV': 'PRP PIV',
               'PRPPR': 'PRP PR',
               'PRPPS': 'PRP PS',
               'PRPS': 'PRP S',
               'PRPS/P': 'PRP S/P',
               'PRPSUBJ': 'PRP SUBJ',
               'PRPVFIN': 'PRP VFIN',
               'SPEC0/1/3S': 'SPEC 0/1/3S',
               'SPEC1': 'SPEC 1',
               'SPEC1/3S': 'SPEC 1/3S',
               'SPEC1P': 'SPEC 1P',
               'SPEC1S': 'SPEC 1S',
               'SPEC2': 'SPEC 2',
               'SPEC2P': 'SPEC 2P',
               'SPEC2S': 'SPEC 2S',
               'SPEC3': 'SPEC 3',
               'SPEC3P': 'SPEC 3P',
               'SPEC3S': 'SPEC 3S',
               'SPECACC': 'SPEC ACC',
               'SPECACC/DAT': 'SPEC ACC/DAT',
               'SPECCOND': 'SPEC COND',
               'SPECDAT': 'SPEC DAT',
               'SPECF': 'SPEC F',
               'SPECFUT': 'SPEC FUT',
               'SPECGER': 'SPEC GER',
               'SPECIMP': 'SPEC IMP',
               'SPECIMPF': 'SPEC IMPF',
               'SPECIND': 'SPEC IND',
               'SPECINF': 'SPEC INF',
               'SPECM': 'SPEC M',
               'SPECM/F': 'SPEC M/F',
               'SPECMQP': 'SPEC MQP',
               'SPECNOM': 'SPEC NOM',
               'SPECP': 'SPEC P',
               'SPECPCP': 'SPEC PCP',
               'SPECPIV': 'SPEC PIV',
               'SPECPR': 'SPEC PR',
               'SPECPS': 'SPEC PS',
               'SPECS': 'SPEC S',
               'SPECS/P': 'SPEC S/P',
               'SPECSUBJ': 'SPEC SUBJ',
               'SPECVFIN': 'SPEC VFIN',
               'V0/1/3S': 'V 0/1/3S',
               'V1': 'V 1',
               'V1/3S': 'V 1/3S',
               'V1P': 'V 1P',
               'V1S': 'V 1S',
               'V2': 'V 2',
               'V2P': 'V 2P',
               'V2S': 'V 2S',
               'V3': 'V 3',
               'V3P': 'V 3P',
               'V3S': 'V 3S',
               'VACC': 'V ACC',
               'VACC/DAT': 'V ACC/DAT',
               'VCOND': 'V COND',
               'VDAT': 'V DAT',
               'VF': 'V F',
               'VFUT': 'V FUT',
               'VGER': 'V GER',
               'VIMP': 'V IMP',
               'VIMPF': 'V IMPF',
               'VIND': 'V IND',
               'VINF': 'V INF',
               'VM': 'V M',
               'VM/F': 'V M/F',
               'VMQP': 'V MQP',
               'VNOM': 'V NOM',
               'VP': 'V P',
               'VPCP': 'V PCP',
               'VPIV': 'V PIV',
               'VPR': 'V PR',
               'VPS': 'V PS',
               'VS': 'V S',
               'VS/P': 'V S/P',
               'VSUBJ': 'V SUBJ',
               'VVFIN': 'V VFIN',}

In [6]:
FILES_ENCODING = {1: 'UTF-8', 2: 'ISO-8859-15'}
PALAVRAS_ENCODING = sys.getfilesystemencoding()
PALAVRAS_PATH = '/opt/palavras/'
base_parser = PALAVRAS_PATH + 'por.pl'
np_parser = PALAVRAS_PATH + 'bin/extract_np.pl'
parser_mode = '--dep'

def palavras_tagger(text):
    #print('Sending to Palavras parser...\n')            
    process = subprocess.Popen([base_parser, parser_mode], 
                               stdin=subprocess.PIPE, 
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE)
    stdout, stderr = process.communicate(text.encode(PALAVRAS_ENCODING))
    text_and_all_tags = []
    text_and_pos_tags = []
    for line in stdout.decode('utf-8').split('\n'):
        line = line.strip() #.decode(PALAVRAS_ENCODING)
        chunks = ''.join([chunk for chunk in line.split() if chunk.startswith('#')])
        if line.isspace() or line == '':
            text_and_all_tags.append(['SPACE','[ignore]','BL','','','',''])
        elif line.startswith('<'):
            text_and_all_tags.append(['STOP','[ignore]','ES','','','',''])
        elif line.startswith('$'):
            non_word = line.split()[0][1:]
            if non_word.isdigit():
                non_word_type = 'NUM'
            else:
                non_word_type = 'NW'
            text_and_all_tags.append([non_word, '[ignore]', non_word_type, '', '', '', chunks])
        elif len(line.split('\t')) < 2:  #Discard malformed lines
            continue
        else:
            info = line.split('\t')
            final = '\t'.join(info[1:]).split()
            word = info[0].strip()    
            lemma = final[0]
            syn_sem_tags = final[1:]
            try:
                pos_tag = ''.join([wc for wc in syn_sem_tags if wc in WORD_CLASSES][0])#avoid picking two Word_Classes
            except IndexError:
                continue
            secondary_tag = ' '.join([sct for sct in syn_sem_tags if sct.startswith('<')])            
            inflexion_tag = ' '.join([it for it in syn_sem_tags if it in INF_TAGS])
            syntactic_tag = ''.join([st for st in syn_sem_tags if st.startswith('@')])
            text_and_all_tags.append([word, lemma, pos_tag, secondary_tag, inflexion_tag, syntactic_tag, chunks])
    num_tokens = len(text_and_all_tags)
    for position in range(num_tokens):
        text_and_pos_tags.append((text_and_all_tags[position][1], text_and_all_tags[position][2]))
    return text_and_all_tags, text_and_pos_tags, stdout #palavras style, nltk style

### pega listas de nomes e ids de documentos que serão usados

In [67]:
names_dataframe_file = os.path.join(inputs,'names_dataframe.pkl')
doc_id_list_file = os.path.join(inputs,'doc_id_list.pkl')

In [339]:
''' caso queira carregar os arquivos '''
names_data = pd.read_pickle(names_dataframe_file)
doc_id_list = pickle.load(open(doc_id_list_file, 'rb'))

In [340]:
names_data.head()

,alt_names,id,source,has_ambiguous_surname,short_fullname,short_surname,important_name
names,,,,,,,
a. a. fisher,"[a. a. fisher, fisher]",104303,frus,False,True,False,False
a. azevedo,"[a. azevedo, azevedo]",500001,cpdoc,True,True,False,False
a. bentinck,"[a. bentinck, bentinck]",101196,frus,False,True,False,False
a. f. susin,"[a. f. susin, susin]",113218,frus,False,True,False,False
a. faria,"[a. faria, faria]",500002,cpdoc,True,True,False,False


# teste, refaz arquivos
muda para csv em vez de pandas df

conserta nomes a partir do pickle

In [341]:
names_data['temp_name'] = names_data.index

In [342]:
names_data[names_data.temp_name.str.contains('arter')]

,alt_names,id,source,has_ambiguous_surname,short_fullname,short_surname,important_name,temp_name
names,,,,,,,,
g. s. carter,"[g. s. carter, carter]",102248,frus,False,True,False,False,g. s. carter
jimmy carter,"[jimmy carter, carter]",500505,cpdoc,False,True,False,True,jimmy carter
rosalynn carter,[rosalynn carter],102258,frus and cpdoc,False,False,False,False,rosalynn carter


In [344]:
outputs = os.path.join('..','outputs')
file_tables = os.path.join(outputs, 'tables_dict.pkl')
cpdoc_tables = pickle.load(open(file_tables, 'rb'))
persons = cpdoc_tables['persons']

In [345]:
persons.head()

,id,name,source,birth_year,death_year,description
0,100002,"Aaron, Harold R.",frus,NaN,NaN,None
1,100005,"Abbas, M. M.",frus,NaN,NaN,None
2,100016,"Abbuhl, Forest e.",frus,NaN,NaN,None
3,100070,"Acheson, Dean Gooderham",None,1893.0,1971.0,Secretary of State from January 1949 until Jan...
4,100096,"Adams, John",frus,NaN,NaN,None


In [346]:
persons[persons['id'] == 500520]

,id,name,source,birth_year,death_year,description


In [347]:
persons[persons.name.str.contains('arter')]

,id,name,source,birth_year,death_year,description
138,102248,"Carter, G. S.",frus,NaN,NaN,None
139,102251,"Carter, James Earl (“Jimmy”), Jr.",None,1924.0,NaN,"President of the United States, from January 2..."
140,102258,"Carter, Rosalynn",frus and cpdoc,NaN,NaN,None
1461,500520,"Carter, Jimmy",cpdoc,NaN,NaN,None


In [332]:
names_data = pd.merge(names_data, persons, on='id', how='left')

In [333]:
names_data[names_data['name'].isnull()]

,alt_names,id,source_x,has_ambiguous_surname,short_fullname,short_surname,important_name,temp_name,name,source_y,birth_year,death_year,description
23,"[adlai e. stevenson, adlai stevenson]",500012,cpdoc,False,False,False,False,adlai e. stevenson,NaN,NaN,NaN,NaN,NaN
192,[araken de oliveira],500113,cpdoc,True,False,False,False,araken de oliveira,NaN,NaN,NaN,NaN,NaN
224,"[ary alcântara, ary alcantara]",500134,cpdoc,True,False,False,False,ary alcântara,NaN,NaN,NaN,NaN,NaN
334,[carlos luz],500200,cpdoc,True,False,True,False,carlos luz,NaN,NaN,NaN,NaN,NaN
337,[carlos manini rios],500203,cpdoc,False,False,True,False,carlos manini rios,NaN,NaN,NaN,NaN,NaN
415,"[cândido rondon, candido rondon]",500252,cpdoc,True,False,False,False,cândido rondon,NaN,NaN,NaN,NaN,NaN
460,[dias leite],500273,cpdoc,True,False,False,False,dias leite,NaN,NaN,NaN,NaN,NaN
462,[dilermando gomes monteiro],500274,cpdoc,True,False,False,False,dilermando gomes monteiro,NaN,NaN,NaN,NaN,NaN
543,[enrique iglesias],500308,cpdoc,False,False,False,False,enrique iglesias,NaN,NaN,NaN,NaN,NaN
546,[enzo debernardi],500309,azeredopapers,False,False,False,False,enzo debernardi,NaN,NaN,NaN,NaN,NaN


In [335]:
nullname_ids = names_data[names_data['name'].isnull()].id.tolist()
nullname_ids[:5]

['500012', '500113', '500134', '500200', '500203']

In [336]:
for i,row in names_data.iterrows():
    if row['id'] not in nullname_ids: continue
#     print(row[7])
    name = ''
    for token in row[0][0].split():
        token = token.capitalize()
        name += token+' '
    name = name.rstrip()
    names_data.at[row.name, 'name'] = name
    print(row.name, name)
names_data.rename({'name': 'names'}, axis=1, inplace=True)
names_data = names_data.replace('(.*),(.*),.*', r'\1,\2', regex=True)
names_data = names_data.replace('(.*), (.*)', r'\2 \1', regex=True)
names_data[names_data['names'].isnull()]

23 Adlai E. Stevenson
192 Araken De Oliveira
224 Ary Alcântara
334 Carlos Luz
337 Carlos Manini Rios
415 Cândido Rondon
460 Dias Leite
462 Dilermando Gomes Monteiro
543 Enrique Iglesias
546 Enzo Debernardi
555 Erik Suy
562 Ernesto Silva
566 Esdras Gueiros
586 Evandro Carreira
708 Gerhard Wolter
830 Hisham El Shawa
848 Humberto Barreto
947 Jarbas Passarinho
1034 Josué De Castro
1044 José Costa
1052 José Eugênio De Macedo Soares
1165 Leite De Castro
1181 Leônidas Sampaio
1225 Luís Rey
1267 Marcelo Damy
1434 Olavo Bilac
1452 Orlando Leite Ribeiro
1457 Oscar Bloch
1458 Oscar Freire
1684 Roberto Alves
1698 Roberto Simonsen
1724 Rubens Resstel
1726 Rubens Vaz
1728 Rui Barbosa
1769 Sebastião Do Rego Barros
1819 Tarcísio Delgado
1821 Tarcísio Meirelles Padilha
1854 V. S. Ivanov
1940 Yasser Arafat
1954 Álvaro Valle


,alt_names,id,source_x,has_ambiguous_surname,short_fullname,short_surname,important_name,temp_name,names,source_y,birth_year,death_year,description


In [338]:
names_data[names_data.names.str.contains('arter')]

,alt_names,id,source_x,has_ambiguous_surname,short_fullname,short_surname,important_name,temp_name,names,source_y,birth_year,death_year,description
665,"[g. s. carter, carter]",102248,frus,False,True,False,False,g. s. carter,G. S. Carter,frus,NaN,NaN,NaN
1035,"[josué montello, josue montello]",500520,cpdoc,False,False,False,False,josué montello,Jimmy Carter,cpdoc,NaN,NaN,NaN
1717,[rosalynn carter],102258,frus and cpdoc,False,False,False,False,rosalynn carter,Rosalynn Carter,frus and cpdoc,NaN,NaN,NaN


In [280]:
names_data.set_index('names', inplace=True)

salva

In [281]:
file_name = os.path.join(inputs, 'names_dataframe.csv')
names_data.to_csv(file_name)

lê de volta

In [282]:
names_data = pd.read_csv(file_name)
names_data = names_data.set_index('names')
names_data

,alt_names,id,source_x,has_ambiguous_surname,short_fullname,short_surname,important_name,source_y,birth_year,death_year,description
names,,,,,,,,,,,
A. A. Fisher,"['a. a. fisher', 'fisher']",104303,frus,False,True,False,False,frus,NaN,NaN,NaN
A. Azevedo,"['a. azevedo', 'azevedo']",500001,cpdoc,True,True,False,False,cpdoc,NaN,NaN,NaN
A. Bentinck,"['a. bentinck', 'bentinck']",101196,frus,False,True,False,False,frus,NaN,NaN,NaN
A. F. Susin,"['a. f. susin', 'susin']",113218,frus,False,True,False,False,frus,NaN,NaN,NaN
A. Faria,"['a. faria', 'faria']",500002,cpdoc,True,True,False,False,cpdoc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
Vasconcelos Costa,"['álvaro cunha', 'alvaro cunha']",500978,cpdoc,True,False,False,False,cpdoc,NaN,NaN,NaN
Vasconcelos Torres,"['álvaro da costa franco', 'alvaro da costa fr...",500979,cpdoc,True,False,False,False,cpdoc,NaN,NaN,NaN
Álvaro Valle,"['álvaro valle', 'alvaro valle']",500980,cpdoc,False,False,False,False,NaN,NaN,NaN,NaN


### faz extração de entidades
conta número de ocorrência de nomes (da lista de nomes) em documentos, com verificação de algumas condições:
* analisamos o tamanho do nome, dos sobrenomes, ambiguidades, etc
* usamos o 'palavras' para analisar se o nome não seria na verdade uma rua (top), cidade (civ) ou organização (org)

In [283]:
names_data.head(3)

,alt_names,id,source_x,has_ambiguous_surname,short_fullname,short_surname,important_name,source_y,birth_year,death_year,description
names,,,,,,,,,,,
A. A. Fisher,"['a. a. fisher', 'fisher']",104303,frus,False,True,False,False,frus,NaN,NaN,NaN
A. Azevedo,"['a. azevedo', 'azevedo']",500001,cpdoc,True,True,False,False,cpdoc,NaN,NaN,NaN
A. Bentinck,"['a. bentinck', 'bentinck']",101196,frus,False,True,False,False,frus,NaN,NaN,NaN


In [284]:
persons_list = []
for row in names_data.itertuples():
    name = row[0]
    person_id = row[2]
    source = row[3]
    person = [person_id, name, source]
    persons_list.append(person)

In [285]:
persons_list[:5]

[[104303, 'A. A. Fisher', 'frus'],
 [500001, 'A. Azevedo', 'cpdoc'],
 [101196, 'A. Bentinck', 'frus'],
 [113218, 'A. F. Susin', 'frus'],
 [500002, 'A. Faria', 'cpdoc']]

### Extração 

In [286]:
len(person_doc_list)

10497

temp files of persons

In [45]:
with open(os.path.join(inputs,'temp-person_doc.pkl'), 'rb') as f:
    person_doc_list = pickle.load(f)

### check for duplicates in persons_list

In [290]:
len(list(zip(*persons_list))[0])

1957

In [291]:
len(set(list(zip(*persons_list))[0]))

1904

In [292]:
unzipped = list(zip(*persons_list))[0]

In [293]:
duplicated_ids = [item for item, count in collections.Counter(unzipped).items() if count > 1]
print('length:', len(duplicated_ids))
print(duplicated_ids)

length: 51
[108408, 115847, 111294, 104698, 106351, 111063, 106700, 103887, 107314, 107024, 111954, 106686, 101368, 105260, 116279, 109171, 109554, 116214, 107252, 111914, 107556, 113142, 103190, 102538, 104322, 107026, 108329, 110885, 111945, 115232, 103879, 100692, 102884, 102333, 110613, 115994, 104230, 101822, 106673, 109882, 112658, 100444, 107032, 116267, 111219, 106300, 114246, 111162, 116152, 102566, 111642]


In [294]:
duplicated = [i for i in persons_list if i[0] in duplicated_ids] 
print('length:',len(duplicated))
duplicated[-10:]

length: 104


[[111162, 'Vicente Rao', 'frus and cpdoc'],
 [111162, 'Vicente Rao', 'frus'],
 [116152, 'William Q. Stanton', 'frus'],
 [109171, 'William K. Miller', 'frus'],
 [109171, 'William K. Miller', 'frus'],
 [116152, 'William Q. Stanton', 'frus'],
 [102566, 'Winston Churchill', 'frus and cpdoc'],
 [102566, 'Winston Churchill', 'frus'],
 [111642, 'Zenon Rossides', 'frus'],
 [111642, 'Zenon Rossides', 'frus and cpdoc']]

#### remove duplicates

In [295]:
ids_list = []
persons_list_clean = []
for i in persons_list:
    if i[0] not in ids_list:
        persons_list_clean.append(i)
        ids_list.append(i[0])
print('length:', len(persons_list_clean))

length: 1904


In [297]:
persons_list_clean[:5]

[[104303, 'A. A. Fisher', 'frus'],
 [500001, 'A. Azevedo', 'cpdoc'],
 [101196, 'A. Bentinck', 'frus'],
 [113218, 'A. F. Susin', 'frus'],
 [500002, 'A. Faria', 'cpdoc']]

### alimenta o banco de dados

In [309]:
sql_db = os.path.join(outputs, 'cpdoc_as.sqlite')
conn = sqlite3.connect(sql_db)
cur = conn.cursor()
schema = '''DROP TABLE IF EXISTS topic_doc2;'''
cur.executescript(schema)

In [308]:
sql_db = os.path.join(outputs, 'cpdoc_as.sqlite')
conn = sqlite3.connect(sql_db)
cur = conn.cursor()

schema = '''
DROP TABLE IF EXISTS persons;
CREATE TABLE IF NOT EXISTS persons
           (id VARCHAR(128) PRIMARY KEY, person_name VARCHAR(128)
           DEFAULT NULL);
DROP TABLE IF EXISTS person_doc;
CREATE TABLE IF NOT EXISTS person_doc
           (person_id VARCHAR(128), doc_id VARCHAR(31), person_count SMALLINT(5),
           FOREIGN KEY(person_id) REFERENCES persons(id)
           FOREIGN KEY(doc_id) REFERENCES docs(id)
           );
'''
cur.executescript(schema)

cur.execute("SELECT * FROM docs WHERE main_language = 'pt' AND (readability > 0.4 OR readability = -1) ")  

text_data = cur.fetchall()

numrows = len(person_doc_list)
percentil = int(numrows/100)

for row in person_doc_list:
    person_id = row[0]
    doc_id = row[1]
    person_count = row[2]
    query = "INSERT INTO person_doc VALUES (?, ?, ?)"
    cur.execute(query, (person_id, doc_id, person_count))

    ### mede percentual de conclusão da tarefa ###
    count = person_doc_list.index(row) + 1
    if count % 100 == 0: 
        clear_output()
        print(int((count)/percentil),'% done on first list')

print('Finished person_doc_list. Wait just a few more minutes to finish dumping data on MySQL.')

for row in persons_list_clean:
    person_id = row[0]
    name = row[1]
    query = "INSERT INTO persons VALUES (?, ?)"
    cur.execute(query, (person_id, name))

conn.commit()
conn.close()
clear_output()
print('All Done')

All Done


# fix sql